### Import Module

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation, LSTM, Dropout, TimeDistributed, Flatten
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.utils import class_weight
from sklearn.utils import class_weight
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflo

### Data Preparation

In [2]:
training_load = pd.read_csv('training_1961_2016.csv', delimiter=',')
testing_2017_load = pd.read_csv('testing_2017.csv', delimiter=',')
testing_2018_load = pd.read_csv('testing_2018.csv', delimiter=',')
testing_2019_load = pd.read_csv('testing_2019.csv', delimiter=',')

In [3]:
training_load.head(5)

,FirstYear_index,FirstYear_playerID,FirstYear_yearID,FirstYear_teamID,FirstYear_lgID,FirstYear_stint,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,...,SixthYear_HBP,SixthYear_SH,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age
0,41261,aaronha01,1961,ML1,NL,1.0,155.0,603.0,115.0,197.0,...,1.0,0.0,8.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,32.0
1,41959,aaronha01,1962,ML1,NL,1.0,156.0,592.0,127.0,191.0,...,0.0,0.0,6.0,11.0,Hank,Aaron,Henry Louis,180.0,72.0,33.0
2,42719,aaronha01,1963,ML1,NL,1.0,161.0,631.0,121.0,201.0,...,1.0,0.0,5.0,21.0,Hank,Aaron,Henry Louis,180.0,72.0,34.0
3,43471,aaronha01,1964,ML1,NL,1.0,145.0,570.0,103.0,187.0,...,2.0,0.0,3.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,35.0
4,44225,aaronha01,1965,ML1,NL,1.0,150.0,570.0,109.0,181.0,...,2.0,0.0,6.0,13.0,Hank,Aaron,Henry Louis,180.0,72.0,36.0


In [4]:
var_name = ['index', 'playerID', 'teamID', 'lgID', 'stint','nameFirst', 'nameLast', 'nameGiven']

First = []
for i in range(len(var_name)):
    First.append(f"FirstYear_{var_name[i]}")

Second = []
for i in range(len(var_name)):
    Second.append(f"SecondYear_{var_name[i]}")

Third = []
for i in range(len(var_name)):
    Third.append(f"ThirdYear_{var_name[i]}")
    
Fourth = []
for i in range(len(var_name)):
    Fourth.append(f"FourthYear_{var_name[i]}")
    
Fifth = []
for i in range(len(var_name)):
    Fifth.append(f"FifthYear_{var_name[i]}")
    
FileColumnName = First + Second + Third + Fourth + Fifth

In [5]:
x_train = training_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(training_load), 5, 21))
y_train = training_load['SixthYear_HR'].values
x_test_2017 = testing_2017_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2017_load), 5, 21))
y_test_2017 = testing_2017_load['SixthYear_HR'].values
x_test_2018 = testing_2018_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2018_load), 5, 21))
y_test_2018 = testing_2018_load['SixthYear_HR'].values
x_test_2019 = testing_2019_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2019_load), 5, 21))
y_test_2019 = testing_2019_load['SixthYear_HR'].values

In [6]:
x_train_2018 = np.concatenate((x_train, x_test_2017, x_test_2018))
y_train_2018 = np.concatenate((y_train, y_test_2017, y_test_2018))

In [78]:
testing_2019 = testing_2019_load.drop(FileColumnName, axis=1).copy()

### Models

In [8]:
model_A = Sequential()
model_A.add(LSTM(128, input_shape=(5, 21), return_sequences=True))
model_A.add(LSTM(128, return_sequences=True))
model_A.add(Dropout(rate=0.5))
model_A.add(Flatten())
model_A.add(Dense(1024, activation="relu"))
model_A.add(Dropout(rate=0.5))
model_A.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_A.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_A.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 5, 128)            76800     
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 128)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              656384    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total para

In [10]:
model_B = Sequential()
model_B.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_B.add(LSTM(16, return_sequences=True))
model_B.add(LSTM(8, return_sequences=True))
model_B.add(Dropout(rate=0.5))
model_B.add(Flatten())
model_B.add(Dense(512, activation="relu"))
model_B.add(Dropout(rate=0.5))
model_B.add(Dense(64, activation="relu"))
model_B.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_B.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_B.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 5, 32)             6912      
_________________________________________________________________
lstm_9 (LSTM)                (None, 5, 16)             3136      
_________________________________________________________________
lstm_10 (LSTM)               (None, 5, 8)              800       
_________________________________________________________________
dropout_7 (Dropout)          (None, 5, 8)              0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 40)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               20992     
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
__________

In [11]:
model_C = Sequential()
model_C.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_C.add(LSTM(32, return_sequences=True))
model_C.add(BatchNormalization())
model_C.add(Flatten())
model_C.add(Dense(512, activation="relu"))
model_C.add(BatchNormalization())
model_C.add(Dense(64, activation="relu"))
model_C.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_C.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_C.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 5, 32)             6912      
_________________________________________________________________
lstm_12 (LSTM)               (None, 5, 32)             8320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 32)             128       
_________________________________________________________________
flatten_5 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               82432     
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_13 (Dense)             (None, 64)                32832     
__________

In [12]:
model_D = Sequential()
model_D.add(LSTM(64, input_shape=(5, 21), return_sequences=True))
model_D.add(LSTM(64, return_sequences=True))
model_D.add(Dropout(rate=0.5))
model_D.add(TimeDistributed(Dense(10)))
model_D.add(Flatten())
model_D.add(Dense(512, activation="relu"))
model_D.add(Dropout(rate=0.5))
model_D.add(Dense(64, activation="relu"))
model_D.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_D.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 5, 64)             22016     
_________________________________________________________________
lstm_14 (LSTM)               (None, 5, 64)             33024     
_________________________________________________________________
dropout_9 (Dropout)          (None, 5, 64)             0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 10)             650       
_________________________________________________________________
flatten_6 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               26112     
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
__________

In [13]:
model_E = Sequential()
model_E.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_E.add(LSTM(32, return_sequences=True))
model_E.add(Dropout(rate=0.5))
model_E.add(Flatten())
model_E.add(Dense(512, activation="relu"))
model_E.add(Dropout(rate=0.5))
model_E.add(Dense(64, activation="relu"))
model_E.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_E.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_E.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 5, 32)             6912      
_________________________________________________________________
lstm_16 (LSTM)               (None, 5, 32)             8320      
_________________________________________________________________
dropout_11 (Dropout)         (None, 5, 32)             0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               82432     
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                32832     
__________

### Training

In [18]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2018)
                                               ,y_train_2018)

filepath = "A/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_A_stat = model_A.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 10012 samples, validate on 191 samples
Epoch 1/1000
10012/10012 [==============================] - 7s 722us/step - loss: 99.6181 - acc: 0.0401 - val_loss: 99.5978 - val_acc: 0.0419

Epoch 00001: val_acc improved from -inf to 0.04188, saving model to B/saved-model-001-0.0419.hdf5
Epoch 2/1000
10012/10012 [==============================] - 6s 611us/step - loss: 68.8713 - acc: 0.0504 - val_loss: 88.3587 - val_acc: 0.0314

Epoch 00002: val_acc did not improve
Epoch 3/1000
10012/10012 [==============================] - 6s 610us/step - loss: 63.6327 - acc: 0.0577 - val_loss: 104.1844 - val_acc: 0.0524

Epoch 00003: val_acc improved from 0.04188 to 0.05236, saving model to B/saved-model-003-0.0524.hdf5
Epoch 4/1000
10012/10012 [==============================] - 7s 651us/step - loss: 62.3286 - acc: 0.0559 - val_loss: 86.2210 - val_acc: 0.0524

Epoch 00004: val_acc improved from 0.05236 to 0.05236, saving model to B/saved-model-004-0.0524.hdf5
Epoch 5/1000
10012/10012 [================


Epoch 00087: val_acc did not improve
Epoch 88/1000
10012/10012 [==============================] - 7s 651us/step - loss: 47.2941 - acc: 0.0753 - val_loss: 89.8609 - val_acc: 0.0681

Epoch 00088: val_acc did not improve
Epoch 89/1000
10012/10012 [==============================] - 6s 609us/step - loss: 47.7444 - acc: 0.0738 - val_loss: 81.6251 - val_acc: 0.0419

Epoch 00089: val_acc did not improve
Epoch 90/1000
10012/10012 [==============================] - 6s 608us/step - loss: 46.9551 - acc: 0.0761 - val_loss: 98.5545 - val_acc: 0.0681

Epoch 00090: val_acc did not improve
Epoch 91/1000
10012/10012 [==============================] - 6s 604us/step - loss: 47.1065 - acc: 0.0780 - val_loss: 79.0656 - val_acc: 0.0419

Epoch 00091: val_acc did not improve
Epoch 92/1000
10012/10012 [==============================] - 6s 608us/step - loss: 47.0073 - acc: 0.0719 - val_loss: 77.1046 - val_acc: 0.0524

Epoch 00092: val_acc did not improve
Epoch 93/1000
10012/10012 [==============================


Epoch 00176: val_acc did not improve
Epoch 177/1000
10012/10012 [==============================] - 6s 613us/step - loss: 46.2987 - acc: 0.0790 - val_loss: 87.1404 - val_acc: 0.0576

Epoch 00177: val_acc did not improve
Epoch 178/1000
10012/10012 [==============================] - 6s 614us/step - loss: 47.9373 - acc: 0.0754 - val_loss: 85.5756 - val_acc: 0.0314

Epoch 00178: val_acc did not improve
Epoch 179/1000
10012/10012 [==============================] - 6s 605us/step - loss: 46.9959 - acc: 0.0730 - val_loss: 80.6043 - val_acc: 0.0471

Epoch 00179: val_acc did not improve
Epoch 180/1000
10012/10012 [==============================] - 6s 619us/step - loss: 46.6918 - acc: 0.0775 - val_loss: 81.0380 - val_acc: 0.0419

Epoch 00180: val_acc did not improve
Epoch 181/1000
10012/10012 [==============================] - 6s 607us/step - loss: 46.9821 - acc: 0.0786 - val_loss: 71.7683 - val_acc: 0.0524

Epoch 00181: val_acc did not improve
Epoch 182/1000
10012/10012 [========================

10012/10012 [==============================] - 6s 625us/step - loss: 47.3255 - acc: 0.0742 - val_loss: 79.9294 - val_acc: 0.0419

Epoch 00266: val_acc did not improve
Epoch 267/1000
10012/10012 [==============================] - 6s 603us/step - loss: 46.3788 - acc: 0.0751 - val_loss: 82.0081 - val_acc: 0.0628

Epoch 00267: val_acc did not improve
Epoch 268/1000
10012/10012 [==============================] - 6s 609us/step - loss: 46.4879 - acc: 0.0766 - val_loss: 83.2065 - val_acc: 0.0471

Epoch 00268: val_acc did not improve
Epoch 269/1000
10012/10012 [==============================] - 6s 639us/step - loss: 46.6736 - acc: 0.0779 - val_loss: 76.9992 - val_acc: 0.0262

Epoch 00269: val_acc did not improve
Epoch 270/1000
10012/10012 [==============================] - 6s 603us/step - loss: 46.5398 - acc: 0.0754 - val_loss: 79.6034 - val_acc: 0.0628

Epoch 00270: val_acc did not improve
Epoch 271/1000
10012/10012 [==============================] - 6s 599us/step - loss: 46.3182 - acc: 0.0774


Epoch 00310: val_acc did not improve
Epoch 311/1000
10012/10012 [==============================] - 6s 602us/step - loss: 46.5563 - acc: 0.0766 - val_loss: 83.8474 - val_acc: 0.0576

Epoch 00311: val_acc did not improve
Epoch 312/1000
10012/10012 [==============================] - 6s 610us/step - loss: 46.2752 - acc: 0.0786 - val_loss: 77.7685 - val_acc: 0.0576

Epoch 00312: val_acc did not improve
Epoch 313/1000
10012/10012 [==============================] - 6s 606us/step - loss: 46.7155 - acc: 0.0709 - val_loss: 79.1758 - val_acc: 0.0366

Epoch 00313: val_acc did not improve
Epoch 314/1000
10012/10012 [==============================] - 6s 618us/step - loss: 46.0858 - acc: 0.0777 - val_loss: 78.8026 - val_acc: 0.0471

Epoch 00314: val_acc did not improve
Epoch 315/1000
10012/10012 [==============================] - 6s 643us/step - loss: 46.0800 - acc: 0.0753 - val_loss: 75.6601 - val_acc: 0.0262

Epoch 00315: val_acc did not improve
Epoch 316/1000
10012/10012 [========================

...

10012/10012 [==============================] - 6s 616us/step - loss: 46.8610 - acc: 0.0719 - val_loss: 89.0666 - val_acc: 0.0576

Epoch 00626: val_acc did not improve
Epoch 627/1000
10012/10012 [==============================] - 6s 636us/step - loss: 45.6476 - acc: 0.0781 - val_loss: 78.7513 - val_acc: 0.0628

Epoch 00627: val_acc did not improve
Epoch 628/1000
10012/10012 [==============================] - 6s 634us/step - loss: 45.2084 - acc: 0.0764 - val_loss: 74.5903 - val_acc: 0.0471

Epoch 00628: val_acc did not improve
Epoch 629/1000
10012/10012 [==============================] - 6s 621us/step - loss: 47.0507 - acc: 0.0730 - val_loss: 82.5423 - val_acc: 0.0576

Epoch 00629: val_acc did not improve
Epoch 630/1000
10012/10012 [==============================] - 6s 632us/step - loss: 45.6526 - acc: 0.0756 - val_loss: 77.5942 - val_acc: 0.0524

Epoch 00630: val_acc did not improve
Epoch 631/1000
10012/10012 [==============================] - 6s 613us/step - loss: 46.0988 - acc: 0.0785

10012/10012 [==============================] - 6s 625us/step - loss: 46.6596 - acc: 0.0748 - val_loss: 77.4496 - val_acc: 0.0471

Epoch 00716: val_acc did not improve
Epoch 717/1000
10012/10012 [==============================] - 6s 637us/step - loss: 45.3504 - acc: 0.0796 - val_loss: 76.8833 - val_acc: 0.0471

Epoch 00717: val_acc did not improve
Epoch 718/1000
10012/10012 [==============================] - 6s 625us/step - loss: 45.2610 - acc: 0.0765 - val_loss: 79.5166 - val_acc: 0.0628

Epoch 00718: val_acc did not improve
Epoch 719/1000
10012/10012 [==============================] - 6s 614us/step - loss: 45.3067 - acc: 0.0761 - val_loss: 75.0587 - val_acc: 0.0471

Epoch 00719: val_acc did not improve
Epoch 720/1000
10012/10012 [==============================] - 6s 616us/step - loss: 45.9750 - acc: 0.0770 - val_loss: 77.5913 - val_acc: 0.0366

Epoch 00720: val_acc did not improve
Epoch 721/1000
10012/10012 [==============================] - 6s 626us/step - loss: 45.2905 - acc: 0.0750

10012/10012 [==============================] - 6s 614us/step - loss: 45.8587 - acc: 0.0785 - val_loss: 75.2189 - val_acc: 0.0314

Epoch 00806: val_acc did not improve
Epoch 807/1000
10012/10012 [==============================] - 6s 612us/step - loss: 45.8682 - acc: 0.0784 - val_loss: 80.5308 - val_acc: 0.0314

Epoch 00807: val_acc did not improve
Epoch 808/1000
10012/10012 [==============================] - 6s 609us/step - loss: 45.0986 - acc: 0.0781 - val_loss: 85.9682 - val_acc: 0.0576

Epoch 00808: val_acc did not improve
Epoch 809/1000
10012/10012 [==============================] - 6s 612us/step - loss: 45.2281 - acc: 0.0774 - val_loss: 80.6773 - val_acc: 0.0524

Epoch 00809: val_acc did not improve
Epoch 810/1000
10012/10012 [==============================] - 6s 615us/step - loss: 45.4088 - acc: 0.0763 - val_loss: 83.5848 - val_acc: 0.0524

Epoch 00810: val_acc did not improve
Epoch 811/1000
10012/10012 [==============================] - 6s 620us/step - loss: 45.7665 - acc: 0.0761


Epoch 00895: val_acc did not improve
Epoch 896/1000
10012/10012 [==============================] - 6s 623us/step - loss: 46.4269 - acc: 0.0795 - val_loss: 87.8420 - val_acc: 0.0471

Epoch 00896: val_acc did not improve
Epoch 897/1000
10012/10012 [==============================] - 7s 708us/step - loss: 45.3436 - acc: 0.0782 - val_loss: 76.3668 - val_acc: 0.0209

Epoch 00897: val_acc did not improve
Epoch 898/1000
10012/10012 [==============================] - 7s 653us/step - loss: 45.1104 - acc: 0.0795 - val_loss: 79.4927 - val_acc: 0.0628

Epoch 00898: val_acc did not improve
Epoch 899/1000
10012/10012 [==============================] - 6s 626us/step - loss: 45.4356 - acc: 0.0794 - val_loss: 81.3491 - val_acc: 0.0524

Epoch 00899: val_acc did not improve
Epoch 900/1000
10012/10012 [==============================] - 6s 620us/step - loss: 46.5914 - acc: 0.0785 - val_loss: 84.6367 - val_acc: 0.0733

Epoch 00900: val_acc did not improve
Epoch 901/1000
10012/10012 [========================

10012/10012 [==============================] - 6s 609us/step - loss: 45.6424 - acc: 0.0758 - val_loss: 79.9414 - val_acc: 0.0733

Epoch 00941: val_acc did not improve
Epoch 942/1000
10012/10012 [==============================] - 6s 607us/step - loss: 45.2697 - acc: 0.0738 - val_loss: 76.1091 - val_acc: 0.0314

Epoch 00942: val_acc did not improve
Epoch 943/1000
10012/10012 [==============================] - 6s 610us/step - loss: 46.9193 - acc: 0.0760 - val_loss: 81.1164 - val_acc: 0.0471

Epoch 00943: val_acc did not improve
Epoch 944/1000
10012/10012 [==============================] - 6s 613us/step - loss: 45.3759 - acc: 0.0754 - val_loss: 80.4795 - val_acc: 0.0419

Epoch 00944: val_acc did not improve
Epoch 945/1000
10012/10012 [==============================] - 6s 611us/step - loss: 44.9277 - acc: 0.0775 - val_loss: 79.6874 - val_acc: 0.0366

Epoch 00945: val_acc did not improve
Epoch 946/1000
10012/10012 [==============================] - 6s 626us/step - loss: 45.1317 - acc: 0.0739

In [20]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2018)
                                               ,y_train_2018)

filepath = "B/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_B_stat = model_B.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 10012 samples, validate on 191 samples
Epoch 1/1000
10012/10012 [==============================] - 3s 284us/step - loss: 135.4939 - acc: 0.0418 - val_loss: 143.6017 - val_acc: 0.0419

Epoch 00001: val_acc improved from -inf to 0.04188, saving model to D/saved-model-001-0.0419.hdf5
Epoch 2/1000
10012/10012 [==============================] - 2s 155us/step - loss: 96.7837 - acc: 0.0384 - val_loss: 142.0671 - val_acc: 0.0419

Epoch 00002: val_acc improved from 0.04188 to 0.04188, saving model to D/saved-model-002-0.0419.hdf5
Epoch 3/1000
10012/10012 [==============================] - 2s 160us/step - loss: 84.9852 - acc: 0.0411 - val_loss: 114.7617 - val_acc: 0.0314

Epoch 00003: val_acc did not improve
Epoch 4/1000
10012/10012 [==============================] - 2s 156us/step - loss: 80.2973 - acc: 0.0443 - val_loss: 107.5578 - val_acc: 0.0157

Epoch 00004: val_acc did not improve
Epoch 5/1000
10012/10012 [==============================] - 2s 152us/step - loss: 73.2844 - acc: 0.043

10012/10012 [==============================] - 2s 153us/step - loss: 58.6030 - acc: 0.0566 - val_loss: 85.8284 - val_acc: 0.0471

Epoch 00044: val_acc did not improve
Epoch 45/1000
10012/10012 [==============================] - 2s 158us/step - loss: 57.7717 - acc: 0.0567 - val_loss: 93.9279 - val_acc: 0.0157

Epoch 00045: val_acc did not improve
Epoch 46/1000
10012/10012 [==============================] - 2s 162us/step - loss: 59.0232 - acc: 0.0536 - val_loss: 90.7140 - val_acc: 0.0262

Epoch 00046: val_acc did not improve
Epoch 47/1000
10012/10012 [==============================] - 2s 161us/step - loss: 57.5971 - acc: 0.0551 - val_loss: 97.0795 - val_acc: 0.0576

Epoch 00047: val_acc did not improve
Epoch 48/1000
10012/10012 [==============================] - 2s 155us/step - loss: 58.2847 - acc: 0.0564 - val_loss: 100.3425 - val_acc: 0.0105

Epoch 00048: val_acc did not improve
Epoch 49/1000
10012/10012 [==============================] - 2s 157us/step - loss: 57.4767 - acc: 0.0561 - v


Epoch 00133: val_acc did not improve
Epoch 134/1000
10012/10012 [==============================] - 2s 157us/step - loss: 53.4935 - acc: 0.0626 - val_loss: 86.4221 - val_acc: 0.0576

Epoch 00134: val_acc did not improve
Epoch 135/1000
10012/10012 [==============================] - 2s 155us/step - loss: 53.5105 - acc: 0.0689 - val_loss: 98.4294 - val_acc: 0.0576

Epoch 00135: val_acc did not improve
Epoch 136/1000
10012/10012 [==============================] - 2s 159us/step - loss: 54.2741 - acc: 0.0609 - val_loss: 84.1108 - val_acc: 0.0366

Epoch 00136: val_acc did not improve
Epoch 137/1000
10012/10012 [==============================] - 2s 166us/step - loss: 53.7845 - acc: 0.0610 - val_loss: 85.6105 - val_acc: 0.0471

Epoch 00137: val_acc did not improve
Epoch 138/1000
10012/10012 [==============================] - 2s 157us/step - loss: 53.7499 - acc: 0.0625 - val_loss: 80.7509 - val_acc: 0.0366

Epoch 00138: val_acc did not improve
Epoch 139/1000
10012/10012 [========================

10012/10012 [==============================] - 2s 152us/step - loss: 53.0714 - acc: 0.0590 - val_loss: 86.1572 - val_acc: 0.0524

Epoch 00179: val_acc did not improve
Epoch 180/1000
10012/10012 [==============================] - 2s 151us/step - loss: 52.2184 - acc: 0.0662 - val_loss: 75.0013 - val_acc: 0.0314

Epoch 00180: val_acc did not improve
Epoch 181/1000
10012/10012 [==============================] - 2s 153us/step - loss: 51.8557 - acc: 0.0695 - val_loss: 74.1668 - val_acc: 0.0524

Epoch 00181: val_acc did not improve
Epoch 182/1000
10012/10012 [==============================] - 2s 154us/step - loss: 52.8531 - acc: 0.0659 - val_loss: 95.8621 - val_acc: 0.0471

Epoch 00182: val_acc did not improve
Epoch 183/1000
10012/10012 [==============================] - 2s 157us/step - loss: 52.6750 - acc: 0.0667 - val_loss: 97.4704 - val_acc: 0.0524

Epoch 00183: val_acc did not improve
Epoch 184/1000
10012/10012 [==============================] - 2s 150us/step - loss: 53.0958 - acc: 0.0624


Epoch 00223: val_acc did not improve
Epoch 224/1000
10012/10012 [==============================] - 2s 152us/step - loss: 51.8146 - acc: 0.0625 - val_loss: 78.1637 - val_acc: 0.0576

Epoch 00224: val_acc did not improve
Epoch 225/1000
10012/10012 [==============================] - 2s 153us/step - loss: 51.1503 - acc: 0.0640 - val_loss: 70.9891 - val_acc: 0.0524

Epoch 00225: val_acc did not improve
Epoch 226/1000
10012/10012 [==============================] - 2s 152us/step - loss: 51.2752 - acc: 0.0662 - val_loss: 78.0119 - val_acc: 0.0785

Epoch 00226: val_acc did not improve
Epoch 227/1000
10012/10012 [==============================] - 2s 152us/step - loss: 51.3812 - acc: 0.0687 - val_loss: 81.3378 - val_acc: 0.0733

Epoch 00227: val_acc did not improve
Epoch 228/1000
10012/10012 [==============================] - 2s 153us/step - loss: 52.8205 - acc: 0.0624 - val_loss: 74.5428 - val_acc: 0.0471

Epoch 00228: val_acc did not improve
Epoch 229/1000
10012/10012 [========================

...

10012/10012 [==============================] - 2s 156us/step - loss: 50.6817 - acc: 0.0640 - val_loss: 85.3063 - val_acc: 0.0366

Epoch 00718: val_acc did not improve
Epoch 719/1000
10012/10012 [==============================] - 2s 152us/step - loss: 48.3467 - acc: 0.0677 - val_loss: 76.9859 - val_acc: 0.0471

Epoch 00719: val_acc did not improve
Epoch 720/1000
10012/10012 [==============================] - 2s 155us/step - loss: 50.0292 - acc: 0.0660 - val_loss: 72.0439 - val_acc: 0.0681

Epoch 00720: val_acc did not improve
Epoch 721/1000
10012/10012 [==============================] - 2s 152us/step - loss: 48.4924 - acc: 0.0685 - val_loss: 79.2291 - val_acc: 0.0681

Epoch 00721: val_acc did not improve
Epoch 722/1000
10012/10012 [==============================] - 2s 173us/step - loss: 47.5734 - acc: 0.0743 - val_loss: 73.3098 - val_acc: 0.0890

Epoch 00722: val_acc did not improve
Epoch 723/1000
10012/10012 [==============================] - 2s 212us/step - loss: 48.1269 - acc: 0.0722

10012/10012 [==============================] - 2s 153us/step - loss: 48.9765 - acc: 0.0717 - val_loss: 72.7768 - val_acc: 0.0733

Epoch 00808: val_acc did not improve
Epoch 809/1000
10012/10012 [==============================] - 2s 152us/step - loss: 47.4950 - acc: 0.0716 - val_loss: 77.0094 - val_acc: 0.0628

Epoch 00809: val_acc did not improve
Epoch 810/1000
10012/10012 [==============================] - 2s 152us/step - loss: 47.9069 - acc: 0.0679 - val_loss: 73.6315 - val_acc: 0.0733

Epoch 00810: val_acc did not improve
Epoch 811/1000
10012/10012 [==============================] - 2s 152us/step - loss: 48.5899 - acc: 0.0674 - val_loss: 72.2188 - val_acc: 0.0419

Epoch 00811: val_acc did not improve
Epoch 812/1000
10012/10012 [==============================] - 2s 153us/step - loss: 48.0905 - acc: 0.0685 - val_loss: 87.3033 - val_acc: 0.0471

Epoch 00812: val_acc did not improve
Epoch 813/1000
10012/10012 [==============================] - 2s 152us/step - loss: 50.3861 - acc: 0.0702

10012/10012 [==============================] - 2s 192us/step - loss: 47.5778 - acc: 0.0661 - val_loss: 76.6643 - val_acc: 0.0681

Epoch 00853: val_acc did not improve
Epoch 854/1000
10012/10012 [==============================] - 2s 152us/step - loss: 47.3938 - acc: 0.0681 - val_loss: 70.5076 - val_acc: 0.0628

Epoch 00854: val_acc did not improve
Epoch 855/1000
10012/10012 [==============================] - 2s 151us/step - loss: 50.4161 - acc: 0.0683 - val_loss: 72.6042 - val_acc: 0.0628

Epoch 00855: val_acc did not improve
Epoch 856/1000
10012/10012 [==============================] - 2s 152us/step - loss: 49.0802 - acc: 0.0665 - val_loss: 76.0317 - val_acc: 0.0838

Epoch 00856: val_acc did not improve
Epoch 857/1000
10012/10012 [==============================] - 2s 191us/step - loss: 47.8022 - acc: 0.0676 - val_loss: 77.2099 - val_acc: 0.0733

Epoch 00857: val_acc did not improve
Epoch 858/1000
10012/10012 [==============================] - 2s 154us/step - loss: 47.4932 - acc: 0.0666


Epoch 00942: val_acc did not improve
Epoch 943/1000
10012/10012 [==============================] - 2s 152us/step - loss: 48.0931 - acc: 0.0705 - val_loss: 81.6374 - val_acc: 0.0681

Epoch 00943: val_acc did not improve
Epoch 944/1000
10012/10012 [==============================] - 2s 154us/step - loss: 47.8821 - acc: 0.0676 - val_loss: 78.8809 - val_acc: 0.0576

Epoch 00944: val_acc did not improve
Epoch 945/1000
10012/10012 [==============================] - 2s 153us/step - loss: 48.6018 - acc: 0.0674 - val_loss: 87.9025 - val_acc: 0.0524

Epoch 00945: val_acc did not improve
Epoch 946/1000
10012/10012 [==============================] - 2s 153us/step - loss: 47.7032 - acc: 0.0713 - val_loss: 79.0697 - val_acc: 0.0576

Epoch 00946: val_acc did not improve
Epoch 947/1000
10012/10012 [==============================] - 2s 155us/step - loss: 48.2440 - acc: 0.0727 - val_loss: 84.5434 - val_acc: 0.0576

Epoch 00947: val_acc did not improve
Epoch 948/1000
10012/10012 [========================

10012/10012 [==============================] - 2s 155us/step - loss: 47.9659 - acc: 0.0642 - val_loss: 104.5632 - val_acc: 0.0576

Epoch 00988: val_acc did not improve
Epoch 989/1000
10012/10012 [==============================] - 2s 155us/step - loss: 47.8726 - acc: 0.0681 - val_loss: 83.7538 - val_acc: 0.0628

Epoch 00989: val_acc did not improve
Epoch 990/1000
10012/10012 [==============================] - 2s 151us/step - loss: 47.9055 - acc: 0.0737 - val_loss: 71.8792 - val_acc: 0.0681

Epoch 00990: val_acc did not improve
Epoch 991/1000
10012/10012 [==============================] - 2s 151us/step - loss: 49.3298 - acc: 0.0678 - val_loss: 80.5674 - val_acc: 0.0681

Epoch 00991: val_acc did not improve
Epoch 992/1000
10012/10012 [==============================] - 2s 155us/step - loss: 48.2669 - acc: 0.0661 - val_loss: 86.2255 - val_acc: 0.0366

Epoch 00992: val_acc did not improve
Epoch 993/1000
10012/10012 [==============================] - 2s 156us/step - loss: 48.9748 - acc: 0.066

In [21]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2018)
                                               ,y_train_2018)

filepath = "C/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_C_stat = model_C.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 10012 samples, validate on 191 samples
Epoch 1/1000
10012/10012 [==============================] - 3s 308us/step - loss: 97.8697 - acc: 0.0720 - val_loss: 107.9518 - val_acc: 0.0628

Epoch 00001: val_acc improved from -inf to 0.06283, saving model to E/saved-model-001-0.0628.hdf5
Epoch 2/1000
10012/10012 [==============================] - 2s 174us/step - loss: 77.0653 - acc: 0.0492 - val_loss: 111.2627 - val_acc: 0.0524

Epoch 00002: val_acc did not improve
Epoch 3/1000
10012/10012 [==============================] - 2s 164us/step - loss: 74.4979 - acc: 0.0533 - val_loss: 108.3469 - val_acc: 0.0366

Epoch 00003: val_acc did not improve
Epoch 4/1000
10012/10012 [==============================] - 2s 162us/step - loss: 72.7938 - acc: 0.0484 - val_loss: 99.0296 - val_acc: 0.0419

Epoch 00004: val_acc did not improve
Epoch 5/1000
10012/10012 [==============================] - 2s 173us/step - loss: 70.3882 - acc: 0.0497 - val_loss: 184.2158 - val_acc: 0.0105

Epoch 00005: val_acc did

10012/10012 [==============================] - 2s 165us/step - loss: 55.8785 - acc: 0.0581 - val_loss: 88.1670 - val_acc: 0.0262

Epoch 00089: val_acc did not improve
Epoch 90/1000
10012/10012 [==============================] - 2s 163us/step - loss: 56.0115 - acc: 0.0551 - val_loss: 93.5137 - val_acc: 0.0524

Epoch 00090: val_acc did not improve
Epoch 91/1000
10012/10012 [==============================] - 2s 166us/step - loss: 55.2753 - acc: 0.0578 - val_loss: 89.3805 - val_acc: 0.0366

Epoch 00091: val_acc did not improve
Epoch 92/1000
10012/10012 [==============================] - 2s 166us/step - loss: 55.6965 - acc: 0.0602 - val_loss: 89.0166 - val_acc: 0.0262

Epoch 00092: val_acc did not improve
Epoch 93/1000
10012/10012 [==============================] - 2s 164us/step - loss: 54.4222 - acc: 0.0643 - val_loss: 87.1337 - val_acc: 0.0419

Epoch 00093: val_acc did not improve
Epoch 94/1000
10012/10012 [==============================] - 2s 164us/step - loss: 56.1359 - acc: 0.0584 - va


Epoch 00178: val_acc did not improve
Epoch 179/1000
10012/10012 [==============================] - 2s 164us/step - loss: 50.9563 - acc: 0.0673 - val_loss: 88.6196 - val_acc: 0.0471

Epoch 00179: val_acc did not improve
Epoch 180/1000
10012/10012 [==============================] - 2s 168us/step - loss: 50.9155 - acc: 0.0646 - val_loss: 88.0106 - val_acc: 0.0314

Epoch 00180: val_acc did not improve
Epoch 181/1000
10012/10012 [==============================] - 2s 172us/step - loss: 50.6937 - acc: 0.0653 - val_loss: 78.8937 - val_acc: 0.0366

Epoch 00181: val_acc did not improve
Epoch 182/1000
10012/10012 [==============================] - 2s 166us/step - loss: 50.6359 - acc: 0.0677 - val_loss: 71.1497 - val_acc: 0.0314

Epoch 00182: val_acc did not improve
Epoch 183/1000
10012/10012 [==============================] - 2s 164us/step - loss: 49.8444 - acc: 0.0617 - val_loss: 69.5981 - val_acc: 0.0314

Epoch 00183: val_acc did not improve
Epoch 184/1000
10012/10012 [========================

10012/10012 [==============================] - 2s 165us/step - loss: 48.6845 - acc: 0.0784 - val_loss: 74.4004 - val_acc: 0.0681

Epoch 00268: val_acc did not improve
Epoch 269/1000
10012/10012 [==============================] - 2s 164us/step - loss: 47.5525 - acc: 0.0765 - val_loss: 257.8304 - val_acc: 0.0681

Epoch 00269: val_acc did not improve
Epoch 270/1000
10012/10012 [==============================] - 2s 163us/step - loss: 49.2793 - acc: 0.0731 - val_loss: 75.2751 - val_acc: 0.0733

Epoch 00270: val_acc did not improve
Epoch 271/1000
10012/10012 [==============================] - 2s 167us/step - loss: 46.8187 - acc: 0.0757 - val_loss: 102.8397 - val_acc: 0.0419

Epoch 00271: val_acc did not improve
Epoch 272/1000
10012/10012 [==============================] - 2s 168us/step - loss: 47.5425 - acc: 0.0733 - val_loss: 69.2151 - val_acc: 0.0785

Epoch 00272: val_acc did not improve
Epoch 273/1000
10012/10012 [==============================] - 2s 167us/step - loss: 47.9390 - acc: 0.07


Epoch 00312: val_acc did not improve
Epoch 313/1000
10012/10012 [==============================] - 2s 166us/step - loss: 108.7224 - acc: 0.0292 - val_loss: 141.9099 - val_acc: 0.0785

Epoch 00313: val_acc did not improve
Epoch 314/1000
10012/10012 [==============================] - 2s 163us/step - loss: 108.7015 - acc: 0.0300 - val_loss: 161.8015 - val_acc: 0.0262

Epoch 00314: val_acc did not improve
Epoch 315/1000
10012/10012 [==============================] - 2s 164us/step - loss: 108.6948 - acc: 0.0292 - val_loss: 230.2079 - val_acc: 0.0314

Epoch 00315: val_acc did not improve
Epoch 316/1000
10012/10012 [==============================] - 2s 166us/step - loss: 108.7049 - acc: 0.0276 - val_loss: 157.1188 - val_acc: 0.0262

Epoch 00316: val_acc did not improve
Epoch 317/1000
10012/10012 [==============================] - 2s 165us/step - loss: 108.7541 - acc: 0.0300 - val_loss: 136.3856 - val_acc: 0.0262

Epoch 00317: val_acc did not improve
Epoch 318/1000
10012/10012 [==============

...

10012/10012 [==============================] - 2s 165us/step - loss: 108.7084 - acc: 0.0297 - val_loss: 360.3194 - val_acc: 0.0209

Epoch 00665: val_acc did not improve
Epoch 666/1000
10012/10012 [==============================] - 2s 166us/step - loss: 108.7798 - acc: 0.0292 - val_loss: 1867.1053 - val_acc: 0.0000e+00

Epoch 00666: val_acc did not improve
Epoch 667/1000
10012/10012 [==============================] - 2s 163us/step - loss: 108.6912 - acc: 0.0292 - val_loss: 3163.6463 - val_acc: 0.0000e+00

Epoch 00667: val_acc did not improve
Epoch 668/1000
10012/10012 [==============================] - 2s 167us/step - loss: 108.7033 - acc: 0.0292 - val_loss: 222.8705 - val_acc: 0.0314

Epoch 00668: val_acc did not improve
Epoch 669/1000
10012/10012 [==============================] - 2s 167us/step - loss: 108.7041 - acc: 0.0283 - val_loss: 169.2524 - val_acc: 0.0471

Epoch 00669: val_acc did not improve
Epoch 670/1000
10012/10012 [==============================] - 2s 168us/step - loss: 1

10012/10012 [==============================] - 2s 162us/step - loss: 108.7690 - acc: 0.0291 - val_loss: 182933.4351 - val_acc: 0.0000e+00

Epoch 00709: val_acc did not improve
Epoch 710/1000
10012/10012 [==============================] - 2s 164us/step - loss: 108.6878 - acc: 0.0292 - val_loss: 18229.5081 - val_acc: 0.0000e+00

Epoch 00710: val_acc did not improve
Epoch 711/1000
10012/10012 [==============================] - 2s 166us/step - loss: 108.7081 - acc: 0.0292 - val_loss: 11114.1720 - val_acc: 0.0000e+00

Epoch 00711: val_acc did not improve
Epoch 712/1000
10012/10012 [==============================] - 2s 165us/step - loss: 108.7370 - acc: 0.0292 - val_loss: 561.6787 - val_acc: 0.0052

Epoch 00712: val_acc did not improve
Epoch 713/1000
10012/10012 [==============================] - 2s 164us/step - loss: 108.6996 - acc: 0.0292 - val_loss: 129.9489 - val_acc: 0.0262

Epoch 00713: val_acc did not improve
Epoch 714/1000
10012/10012 [==============================] - 2s 166us/step 

10012/10012 [==============================] - 2s 162us/step - loss: 108.7216 - acc: 0.0299 - val_loss: 360.3194 - val_acc: 0.0209

Epoch 00797: val_acc did not improve
Epoch 798/1000
10012/10012 [==============================] - 2s 163us/step - loss: 108.7156 - acc: 0.0292 - val_loss: 204.2775 - val_acc: 0.0314

Epoch 00798: val_acc did not improve
Epoch 799/1000
10012/10012 [==============================] - 2s 164us/step - loss: 108.6722 - acc: 0.0308 - val_loss: 11072.4866 - val_acc: 0.0000e+00

Epoch 00799: val_acc did not improve
Epoch 800/1000
10012/10012 [==============================] - 2s 172us/step - loss: 108.7464 - acc: 0.0274 - val_loss: 5933.0095 - val_acc: 0.0000e+00

Epoch 00800: val_acc did not improve
Epoch 801/1000
10012/10012 [==============================] - 2s 167us/step - loss: 108.8636 - acc: 0.0296 - val_loss: 4158.4686 - val_acc: 0.0000e+00

Epoch 00801: val_acc did not improve
Epoch 802/1000
10012/10012 [==============================] - 2s 165us/step - l

10012/10012 [==============================] - 2s 171us/step - loss: 108.6875 - acc: 0.0294 - val_loss: 360.3194 - val_acc: 0.0209

Epoch 00885: val_acc did not improve
Epoch 886/1000
10012/10012 [==============================] - 2s 165us/step - loss: 108.7812 - acc: 0.0295 - val_loss: 243.2716 - val_acc: 0.0105

Epoch 00886: val_acc did not improve
Epoch 887/1000
10012/10012 [==============================] - 2s 164us/step - loss: 108.7196 - acc: 0.0292 - val_loss: 674.0435 - val_acc: 0.0052

Epoch 00887: val_acc did not improve
Epoch 888/1000
10012/10012 [==============================] - 2s 164us/step - loss: 108.6951 - acc: 0.0292 - val_loss: 1266.1142 - val_acc: 0.0052

Epoch 00888: val_acc did not improve
Epoch 889/1000
10012/10012 [==============================] - 2s 163us/step - loss: 108.6932 - acc: 0.0292 - val_loss: 8986.3139 - val_acc: 0.0000e+00

Epoch 00889: val_acc did not improve
Epoch 890/1000
10012/10012 [==============================] - 2s 166us/step - loss: 108.7

10012/10012 [==============================] - 2s 165us/step - loss: 108.7567 - acc: 0.0292 - val_loss: 360.3194 - val_acc: 0.0209

Epoch 00973: val_acc did not improve
Epoch 974/1000
10012/10012 [==============================] - 2s 166us/step - loss: 108.7851 - acc: 0.0294 - val_loss: 140.9023 - val_acc: 0.0157

Epoch 00974: val_acc did not improve
Epoch 975/1000
10012/10012 [==============================] - 2s 162us/step - loss: 108.7870 - acc: 0.0302 - val_loss: 1868.4000 - val_acc: 0.0000e+00

Epoch 00975: val_acc did not improve
Epoch 976/1000
10012/10012 [==============================] - 2s 162us/step - loss: 108.7520 - acc: 0.0306 - val_loss: 1278.7063 - val_acc: 0.0052

Epoch 00976: val_acc did not improve
Epoch 977/1000
10012/10012 [==============================] - 2s 164us/step - loss: 108.6946 - acc: 0.0292 - val_loss: 2597.9164 - val_acc: 0.0000e+00

Epoch 00977: val_acc did not improve
Epoch 978/1000
10012/10012 [==============================] - 2s 164us/step - loss: 

In [22]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2018)
                                               ,y_train_2018)

filepath = "D/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_D_stat = model_D.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 10012 samples, validate on 191 samples
Epoch 1/1000
10012/10012 [==============================] - 3s 337us/step - loss: 120.3569 - acc: 0.0404 - val_loss: 158.1254 - val_acc: 0.0366

Epoch 00001: val_acc improved from -inf to 0.03665, saving model to F/saved-model-001-0.0366.hdf5
Epoch 2/1000
10012/10012 [==============================] - 2s 201us/step - loss: 91.8064 - acc: 0.0359 - val_loss: 143.8060 - val_acc: 0.0576

Epoch 00002: val_acc improved from 0.03665 to 0.05759, saving model to F/saved-model-002-0.0576.hdf5
Epoch 3/1000
10012/10012 [==============================] - 2s 194us/step - loss: 72.7551 - acc: 0.0516 - val_loss: 105.9073 - val_acc: 0.0681

Epoch 00003: val_acc improved from 0.05759 to 0.06806, saving model to F/saved-model-003-0.0681.hdf5
Epoch 4/1000
10012/10012 [==============================] - 2s 186us/step - loss: 64.5904 - acc: 0.0556 - val_loss: 93.5088 - val_acc: 0.0628

Epoch 00004: val_acc did not improve
Epoch 5/1000
10012/10012 [=============


Epoch 00044: val_acc did not improve
Epoch 45/1000
10012/10012 [==============================] - 2s 221us/step - loss: 50.4412 - acc: 0.0699 - val_loss: 87.0816 - val_acc: 0.0419

Epoch 00045: val_acc did not improve
Epoch 46/1000
10012/10012 [==============================] - 2s 186us/step - loss: 49.8265 - acc: 0.0662 - val_loss: 89.7756 - val_acc: 0.0366

Epoch 00046: val_acc did not improve
Epoch 47/1000
10012/10012 [==============================] - 2s 186us/step - loss: 49.6579 - acc: 0.0698 - val_loss: 101.9678 - val_acc: 0.0890

Epoch 00047: val_acc improved from 0.07853 to 0.08901, saving model to F/saved-model-047-0.0890.hdf5
Epoch 48/1000
10012/10012 [==============================] - 2s 187us/step - loss: 49.3980 - acc: 0.0736 - val_loss: 87.4643 - val_acc: 0.0471

Epoch 00048: val_acc did not improve
Epoch 49/1000
10012/10012 [==============================] - 2s 189us/step - loss: 50.2744 - acc: 0.0755 - val_loss: 96.1006 - val_acc: 0.0471

Epoch 00049: val_acc did not 

10012/10012 [==============================] - 2s 195us/step - loss: 47.4538 - acc: 0.0729 - val_loss: 81.6272 - val_acc: 0.0576

Epoch 00134: val_acc did not improve
Epoch 135/1000
10012/10012 [==============================] - 2s 187us/step - loss: 47.3265 - acc: 0.0783 - val_loss: 93.2470 - val_acc: 0.0838

Epoch 00135: val_acc did not improve
Epoch 136/1000
10012/10012 [==============================] - 2s 182us/step - loss: 47.8607 - acc: 0.0740 - val_loss: 84.6209 - val_acc: 0.0524

Epoch 00136: val_acc did not improve
Epoch 137/1000
10012/10012 [==============================] - 2s 188us/step - loss: 47.3472 - acc: 0.0737 - val_loss: 79.0117 - val_acc: 0.0471

Epoch 00137: val_acc did not improve
Epoch 138/1000
10012/10012 [==============================] - 2s 185us/step - loss: 48.5135 - acc: 0.0710 - val_loss: 81.1927 - val_acc: 0.0838

Epoch 00138: val_acc did not improve
Epoch 139/1000
10012/10012 [==============================] - 2s 185us/step - loss: 47.6159 - acc: 0.0717

10012/10012 [==============================] - 2s 185us/step - loss: 47.7436 - acc: 0.0713 - val_loss: 85.2031 - val_acc: 0.0838

Epoch 00224: val_acc did not improve
Epoch 225/1000
10012/10012 [==============================] - 2s 186us/step - loss: 47.4739 - acc: 0.0699 - val_loss: 87.0222 - val_acc: 0.0681

Epoch 00225: val_acc did not improve
Epoch 226/1000
10012/10012 [==============================] - 2s 186us/step - loss: 48.3972 - acc: 0.0715 - val_loss: 84.6273 - val_acc: 0.0785

Epoch 00226: val_acc did not improve
Epoch 227/1000
10012/10012 [==============================] - 2s 189us/step - loss: 47.6216 - acc: 0.0731 - val_loss: 78.4503 - val_acc: 0.0628

Epoch 00227: val_acc did not improve
Epoch 228/1000
10012/10012 [==============================] - 2s 182us/step - loss: 47.9887 - acc: 0.0700 - val_loss: 78.9875 - val_acc: 0.0366

Epoch 00228: val_acc did not improve
Epoch 229/1000
10012/10012 [==============================] - 2s 181us/step - loss: 47.2925 - acc: 0.0703

10012/10012 [==============================] - 2s 189us/step - loss: 47.5994 - acc: 0.0729 - val_loss: 90.8386 - val_acc: 0.0785

Epoch 00314: val_acc did not improve
Epoch 315/1000
10012/10012 [==============================] - 2s 204us/step - loss: 47.9726 - acc: 0.0678 - val_loss: 86.6212 - val_acc: 0.0576

Epoch 00315: val_acc did not improve
Epoch 316/1000
10012/10012 [==============================] - 2s 186us/step - loss: 47.2022 - acc: 0.0748 - val_loss: 91.4692 - val_acc: 0.0838

Epoch 00316: val_acc did not improve
Epoch 317/1000
10012/10012 [==============================] - 2s 181us/step - loss: 47.4253 - acc: 0.0729 - val_loss: 85.7939 - val_acc: 0.0628

Epoch 00317: val_acc did not improve
Epoch 318/1000
10012/10012 [==============================] - 2s 185us/step - loss: 46.4892 - acc: 0.0754 - val_loss: 96.3485 - val_acc: 0.0785

Epoch 00318: val_acc did not improve
Epoch 319/1000
10012/10012 [==============================] - 2s 185us/step - loss: 47.3305 - acc: 0.0726

...

10012/10012 [==============================] - 2s 181us/step - loss: 46.5938 - acc: 0.0778 - val_loss: 79.1746 - val_acc: 0.0681

Epoch 00584: val_acc did not improve
Epoch 585/1000
10012/10012 [==============================] - 2s 181us/step - loss: 45.5314 - acc: 0.0785 - val_loss: 78.5475 - val_acc: 0.0314

Epoch 00585: val_acc did not improve
Epoch 586/1000
10012/10012 [==============================] - 2s 182us/step - loss: 45.9633 - acc: 0.0728 - val_loss: 87.4592 - val_acc: 0.0471

Epoch 00586: val_acc did not improve
Epoch 587/1000
10012/10012 [==============================] - 2s 190us/step - loss: 47.1938 - acc: 0.0759 - val_loss: 77.4208 - val_acc: 0.0471

Epoch 00587: val_acc did not improve
Epoch 588/1000
10012/10012 [==============================] - 2s 183us/step - loss: 47.5530 - acc: 0.0762 - val_loss: 84.3537 - val_acc: 0.0471

Epoch 00588: val_acc did not improve
Epoch 589/1000
10012/10012 [==============================] - 2s 182us/step - loss: 46.1638 - acc: 0.0691

10012/10012 [==============================] - 2s 192us/step - loss: 45.6889 - acc: 0.0805 - val_loss: 81.9439 - val_acc: 0.0471

Epoch 00674: val_acc did not improve
Epoch 675/1000
10012/10012 [==============================] - 2s 185us/step - loss: 46.0964 - acc: 0.0792 - val_loss: 86.8543 - val_acc: 0.0838

Epoch 00675: val_acc did not improve
Epoch 676/1000
10012/10012 [==============================] - 2s 201us/step - loss: 45.4409 - acc: 0.0797 - val_loss: 86.7035 - val_acc: 0.0576

Epoch 00676: val_acc did not improve
Epoch 677/1000
10012/10012 [==============================] - 2s 190us/step - loss: 45.0249 - acc: 0.0795 - val_loss: 85.8047 - val_acc: 0.0681

Epoch 00677: val_acc did not improve
Epoch 678/1000
10012/10012 [==============================] - 2s 182us/step - loss: 45.7722 - acc: 0.0762 - val_loss: 77.9825 - val_acc: 0.0366

Epoch 00678: val_acc did not improve
Epoch 679/1000
10012/10012 [==============================] - 2s 181us/step - loss: 46.9049 - acc: 0.0749

10012/10012 [==============================] - 2s 193us/step - loss: 47.4373 - acc: 0.0736 - val_loss: 78.5862 - val_acc: 0.0733

Epoch 00764: val_acc did not improve
Epoch 765/1000
10012/10012 [==============================] - 2s 194us/step - loss: 45.9357 - acc: 0.0732 - val_loss: 82.9639 - val_acc: 0.0628

Epoch 00765: val_acc did not improve
Epoch 766/1000
10012/10012 [==============================] - 2s 193us/step - loss: 45.8811 - acc: 0.0783 - val_loss: 84.9012 - val_acc: 0.0733

Epoch 00766: val_acc did not improve
Epoch 767/1000
10012/10012 [==============================] - 2s 193us/step - loss: 46.2971 - acc: 0.0743 - val_loss: 87.3825 - val_acc: 0.0419

Epoch 00767: val_acc did not improve
Epoch 768/1000
10012/10012 [==============================] - 2s 200us/step - loss: 45.5526 - acc: 0.0766 - val_loss: 81.1483 - val_acc: 0.0262

Epoch 00768: val_acc did not improve
Epoch 769/1000
10012/10012 [==============================] - 2s 188us/step - loss: 46.2139 - acc: 0.0742

10012/10012 [==============================] - 2s 196us/step - loss: 45.8911 - acc: 0.0748 - val_loss: 79.5557 - val_acc: 0.0628

Epoch 00854: val_acc did not improve
Epoch 855/1000
10012/10012 [==============================] - 2s 202us/step - loss: 46.4442 - acc: 0.0767 - val_loss: 77.7598 - val_acc: 0.0419

Epoch 00855: val_acc did not improve
Epoch 856/1000
10012/10012 [==============================] - 2s 216us/step - loss: 46.7419 - acc: 0.0648 - val_loss: 80.0761 - val_acc: 0.0576

Epoch 00856: val_acc did not improve
Epoch 857/1000
10012/10012 [==============================] - 2s 189us/step - loss: 46.7964 - acc: 0.0770 - val_loss: 77.4801 - val_acc: 0.0524

Epoch 00857: val_acc did not improve
Epoch 858/1000
10012/10012 [==============================] - 2s 216us/step - loss: 45.8297 - acc: 0.0781 - val_loss: 83.0812 - val_acc: 0.0471

Epoch 00858: val_acc did not improve
Epoch 859/1000
10012/10012 [==============================] - 2s 195us/step - loss: 46.0318 - acc: 0.0770

10012/10012 [==============================] - 2s 183us/step - loss: 45.6580 - acc: 0.0775 - val_loss: 86.7384 - val_acc: 0.0576

Epoch 00944: val_acc did not improve
Epoch 945/1000
10012/10012 [==============================] - 2s 192us/step - loss: 45.7870 - acc: 0.0769 - val_loss: 77.7271 - val_acc: 0.0576

Epoch 00945: val_acc did not improve
Epoch 946/1000
10012/10012 [==============================] - 2s 209us/step - loss: 45.7093 - acc: 0.0717 - val_loss: 85.3403 - val_acc: 0.0524

Epoch 00946: val_acc did not improve
Epoch 947/1000
10012/10012 [==============================] - 2s 198us/step - loss: 45.8553 - acc: 0.0756 - val_loss: 80.5803 - val_acc: 0.0419

Epoch 00947: val_acc did not improve
Epoch 948/1000
10012/10012 [==============================] - 2s 194us/step - loss: 44.8721 - acc: 0.0745 - val_loss: 81.4502 - val_acc: 0.0314

Epoch 00948: val_acc did not improve
Epoch 949/1000
10012/10012 [==============================] - 2s 210us/step - loss: 44.9318 - acc: 0.0758

In [23]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2018)
                                               ,y_train_2018)

filepath = "E/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_E_stat = model_E.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 10012 samples, validate on 191 samples
Epoch 1/1000
10012/10012 [==============================] - 3s 335us/step - loss: 114.4305 - acc: 0.0423 - val_loss: 129.1414 - val_acc: 0.0209

Epoch 00001: val_acc improved from -inf to 0.02094, saving model to G/saved-model-001-0.0209.hdf5
Epoch 2/1000
10012/10012 [==============================] - 1s 143us/step - loss: 74.1385 - acc: 0.0486 - val_loss: 111.7868 - val_acc: 0.0524

Epoch 00002: val_acc improved from 0.02094 to 0.05236, saving model to G/saved-model-002-0.0524.hdf5
Epoch 3/1000
10012/10012 [==============================] - 1s 146us/step - loss: 67.0715 - acc: 0.0545 - val_loss: 98.4239 - val_acc: 0.0628

Epoch 00003: val_acc improved from 0.05236 to 0.06283, saving model to G/saved-model-003-0.0628.hdf5
Epoch 4/1000
10012/10012 [==============================] - 1s 146us/step - loss: 63.5592 - acc: 0.0584 - val_loss: 97.1151 - val_acc: 0.0890

Epoch 00004: val_acc improved from 0.06283 to 0.08901, saving model to G/save

10012/10012 [==============================] - 1s 142us/step - loss: 47.8219 - acc: 0.0773 - val_loss: 85.9780 - val_acc: 0.0838

Epoch 00089: val_acc did not improve
Epoch 90/1000
10012/10012 [==============================] - 1s 143us/step - loss: 48.1058 - acc: 0.0773 - val_loss: 82.5794 - val_acc: 0.0628

Epoch 00090: val_acc did not improve
Epoch 91/1000
10012/10012 [==============================] - 1s 140us/step - loss: 48.0913 - acc: 0.0723 - val_loss: 85.4390 - val_acc: 0.0314

Epoch 00091: val_acc did not improve
Epoch 92/1000
10012/10012 [==============================] - 1s 139us/step - loss: 47.3802 - acc: 0.0794 - val_loss: 82.1933 - val_acc: 0.0419

Epoch 00092: val_acc did not improve
Epoch 93/1000
10012/10012 [==============================] - 1s 144us/step - loss: 49.1001 - acc: 0.0712 - val_loss: 80.5776 - val_acc: 0.0681

Epoch 00093: val_acc did not improve
Epoch 94/1000
10012/10012 [==============================] - 1s 137us/step - loss: 48.2837 - acc: 0.0816 - va

10012/10012 [==============================] - 1s 143us/step - loss: 46.3365 - acc: 0.0774 - val_loss: 86.2764 - val_acc: 0.0628

Epoch 00179: val_acc did not improve
Epoch 180/1000
10012/10012 [==============================] - 2s 157us/step - loss: 47.3392 - acc: 0.0812 - val_loss: 76.2906 - val_acc: 0.0681

Epoch 00180: val_acc did not improve
Epoch 181/1000
10012/10012 [==============================] - 1s 140us/step - loss: 46.7148 - acc: 0.0770 - val_loss: 78.6130 - val_acc: 0.0681

Epoch 00181: val_acc did not improve
Epoch 182/1000
10012/10012 [==============================] - 1s 136us/step - loss: 47.0971 - acc: 0.0800 - val_loss: 83.3870 - val_acc: 0.0785

Epoch 00182: val_acc did not improve
Epoch 183/1000
10012/10012 [==============================] - 1s 136us/step - loss: 46.3282 - acc: 0.0790 - val_loss: 80.4440 - val_acc: 0.0576

Epoch 00183: val_acc did not improve
Epoch 184/1000
10012/10012 [==============================] - 1s 135us/step - loss: 46.0986 - acc: 0.0813


Epoch 00223: val_acc did not improve
Epoch 224/1000
10012/10012 [==============================] - 1s 143us/step - loss: 46.4235 - acc: 0.0753 - val_loss: 78.8197 - val_acc: 0.0681

Epoch 00224: val_acc did not improve
Epoch 225/1000
10012/10012 [==============================] - 1s 138us/step - loss: 46.4064 - acc: 0.0826 - val_loss: 81.3931 - val_acc: 0.0576

Epoch 00225: val_acc did not improve
Epoch 226/1000
10012/10012 [==============================] - 2s 152us/step - loss: 47.2561 - acc: 0.0766 - val_loss: 75.9910 - val_acc: 0.0576

Epoch 00226: val_acc did not improve
Epoch 227/1000
10012/10012 [==============================] - 1s 137us/step - loss: 47.2163 - acc: 0.0769 - val_loss: 84.0370 - val_acc: 0.0628

Epoch 00227: val_acc did not improve
Epoch 228/1000
10012/10012 [==============================] - 1s 138us/step - loss: 46.6324 - acc: 0.0785 - val_loss: 85.6737 - val_acc: 0.0838

Epoch 00228: val_acc did not improve
Epoch 229/1000
10012/10012 [========================

10012/10012 [==============================] - 1s 147us/step - loss: 46.4080 - acc: 0.0779 - val_loss: 78.5242 - val_acc: 0.0628

Epoch 00269: val_acc did not improve
Epoch 270/1000
10012/10012 [==============================] - 1s 139us/step - loss: 46.3321 - acc: 0.0777 - val_loss: 76.3690 - val_acc: 0.0628

Epoch 00270: val_acc did not improve
Epoch 271/1000
10012/10012 [==============================] - 1s 140us/step - loss: 46.8815 - acc: 0.0733 - val_loss: 79.9608 - val_acc: 0.0524

Epoch 00271: val_acc did not improve
Epoch 272/1000
10012/10012 [==============================] - 1s 141us/step - loss: 47.0652 - acc: 0.0729 - val_loss: 83.0876 - val_acc: 0.0628

Epoch 00272: val_acc did not improve
Epoch 273/1000
10012/10012 [==============================] - 1s 140us/step - loss: 45.7574 - acc: 0.0803 - val_loss: 76.8509 - val_acc: 0.0314

Epoch 00273: val_acc did not improve
Epoch 274/1000
10012/10012 [==============================] - 1s 139us/step - loss: 48.2965 - acc: 0.0702

...

10012/10012 [==============================] - 1s 139us/step - loss: 43.7135 - acc: 0.0822 - val_loss: 82.7976 - val_acc: 0.0785

Epoch 00629: val_acc did not improve
Epoch 630/1000
10012/10012 [==============================] - 1s 137us/step - loss: 44.9359 - acc: 0.0801 - val_loss: 85.0521 - val_acc: 0.0628

Epoch 00630: val_acc did not improve
Epoch 631/1000
10012/10012 [==============================] - 1s 137us/step - loss: 43.6617 - acc: 0.0854 - val_loss: 84.5307 - val_acc: 0.0366

Epoch 00631: val_acc did not improve
Epoch 632/1000
10012/10012 [==============================] - 1s 138us/step - loss: 43.8079 - acc: 0.0795 - val_loss: 79.7651 - val_acc: 0.0628

Epoch 00632: val_acc did not improve
Epoch 633/1000
10012/10012 [==============================] - 1s 139us/step - loss: 44.0940 - acc: 0.0811 - val_loss: 80.7942 - val_acc: 0.0471

Epoch 00633: val_acc did not improve
Epoch 634/1000
10012/10012 [==============================] - 1s 142us/step - loss: 43.8901 - acc: 0.0785


Epoch 00673: val_acc did not improve
Epoch 674/1000
10012/10012 [==============================] - 1s 140us/step - loss: 44.3511 - acc: 0.0814 - val_loss: 83.4902 - val_acc: 0.0628

Epoch 00674: val_acc did not improve
Epoch 675/1000
10012/10012 [==============================] - 1s 141us/step - loss: 44.0682 - acc: 0.0770 - val_loss: 85.9476 - val_acc: 0.0733

Epoch 00675: val_acc did not improve
Epoch 676/1000
10012/10012 [==============================] - 1s 137us/step - loss: 43.7383 - acc: 0.0840 - val_loss: 81.1661 - val_acc: 0.0785

Epoch 00676: val_acc did not improve
Epoch 677/1000
10012/10012 [==============================] - 1s 137us/step - loss: 44.2874 - acc: 0.0823 - val_loss: 80.4971 - val_acc: 0.0681

Epoch 00677: val_acc did not improve
Epoch 678/1000
10012/10012 [==============================] - 1s 137us/step - loss: 43.8710 - acc: 0.0820 - val_loss: 88.5016 - val_acc: 0.0681

Epoch 00678: val_acc did not improve
Epoch 679/1000
10012/10012 [========================

10012/10012 [==============================] - 1s 139us/step - loss: 43.1559 - acc: 0.0815 - val_loss: 80.9411 - val_acc: 0.0576

Epoch 00763: val_acc did not improve
Epoch 764/1000
10012/10012 [==============================] - 1s 137us/step - loss: 43.5924 - acc: 0.0850 - val_loss: 79.9933 - val_acc: 0.0733

Epoch 00764: val_acc did not improve
Epoch 765/1000
10012/10012 [==============================] - 1s 140us/step - loss: 43.1317 - acc: 0.0801 - val_loss: 82.6933 - val_acc: 0.0681

Epoch 00765: val_acc did not improve
Epoch 766/1000
10012/10012 [==============================] - 1s 138us/step - loss: 42.7879 - acc: 0.0842 - val_loss: 84.8171 - val_acc: 0.0576

Epoch 00766: val_acc did not improve
Epoch 767/1000
10012/10012 [==============================] - 1s 142us/step - loss: 43.0911 - acc: 0.0825 - val_loss: 83.7530 - val_acc: 0.0471

Epoch 00767: val_acc did not improve
Epoch 768/1000
10012/10012 [==============================] - 1s 140us/step - loss: 43.7332 - acc: 0.0786

10012/10012 [==============================] - 1s 141us/step - loss: 42.4990 - acc: 0.0790 - val_loss: 77.4609 - val_acc: 0.0681

Epoch 00853: val_acc did not improve
Epoch 854/1000
10012/10012 [==============================] - 1s 140us/step - loss: 42.7968 - acc: 0.0829 - val_loss: 79.1876 - val_acc: 0.0576

Epoch 00854: val_acc did not improve
Epoch 855/1000
10012/10012 [==============================] - 1s 137us/step - loss: 42.6485 - acc: 0.0821 - val_loss: 76.7768 - val_acc: 0.0628

Epoch 00855: val_acc did not improve
Epoch 856/1000
10012/10012 [==============================] - 1s 144us/step - loss: 43.8375 - acc: 0.0761 - val_loss: 77.9192 - val_acc: 0.0419

Epoch 00856: val_acc did not improve
Epoch 857/1000
10012/10012 [==============================] - 1s 143us/step - loss: 43.3166 - acc: 0.0827 - val_loss: 86.9260 - val_acc: 0.0314

Epoch 00857: val_acc did not improve
Epoch 858/1000
10012/10012 [==============================] - 1s 149us/step - loss: 43.3503 - acc: 0.0807

10012/10012 [==============================] - 1s 142us/step - loss: 41.9710 - acc: 0.0785 - val_loss: 79.0709 - val_acc: 0.0681

Epoch 00943: val_acc did not improve
Epoch 944/1000
10012/10012 [==============================] - 1s 136us/step - loss: 43.0064 - acc: 0.0790 - val_loss: 84.6184 - val_acc: 0.0576

Epoch 00944: val_acc did not improve
Epoch 945/1000
10012/10012 [==============================] - 1s 139us/step - loss: 42.0546 - acc: 0.0833 - val_loss: 79.7519 - val_acc: 0.0314

Epoch 00945: val_acc did not improve
Epoch 946/1000
10012/10012 [==============================] - 1s 138us/step - loss: 42.8755 - acc: 0.0776 - val_loss: 81.4013 - val_acc: 0.0471

Epoch 00946: val_acc did not improve
Epoch 947/1000
10012/10012 [==============================] - 1s 139us/step - loss: 43.3239 - acc: 0.0855 - val_loss: 84.6869 - val_acc: 0.0576

Epoch 00947: val_acc did not improve
Epoch 948/1000
10012/10012 [==============================] - 1s 139us/step - loss: 41.9451 - acc: 0.0780

### Analysis

In [7]:
AAA = load_model('2019/saved-model-851-0.0942-A.hdf5')
BBB = load_model('2019/saved-model-637-0.1257-B.hdf5')
CCC = load_model('2019/saved-model-203-0.0995-C.hdf5')
DDD = load_model('2019/saved-model-109-0.1099-D.hdf5')
EEE = load_model('2019/saved-model-714-0.1152-E.hdf5')

In [87]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = AAA.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2019)))
print("正負1：%.4f" % (a/len(y_test_2019)))
print("正負3：%.4f" % (b/len(y_test_2019)))
print("正負5：%.4f" % (c/len(y_test_2019)))
print("正負10：%.4f" % (d/len(y_test_2019)))

答對：0.0942
正負1：0.1675
正負3：0.3560
正負5：0.5183
正負10：0.7592


In [88]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.721404946287265
mean_squared_error: 77.85735441543807
rmse: 8.823681454780543


In [89]:
Counter(sorted(p))

Counter({2.0: 2,
         4.0: 1,
         5.0: 1,
         6.0: 1,
         7.0: 3,
         9.0: 1,
         10.0: 1,
         12.0: 2,
         13.0: 2,
         14.0: 1,
         20.0: 1,
         34.0: 1,
         35.0: 1})

In [90]:
Counter(sorted(n))

Counter({1.0: 1,
         2.0: 3,
         3.0: 3,
         4.0: 1,
         5.0: 2,
         6.0: 2,
         7.0: 3,
         9.0: 2,
         10.0: 1,
         11.0: 1,
         12.0: 3,
         13.0: 2,
         14.0: 1,
         16.0: 1,
         17.0: 1,
         20.0: 1,
         25.0: 1,
         34.0: 2,
         35.0: 1})

In [91]:
Counter(sorted(m))

Counter({0.0: 2,
         1.0: 4,
         2.0: 5,
         3.0: 3,
         4.0: 1,
         5.0: 6,
         6.0: 3,
         7.0: 5,
         8.0: 2,
         9.0: 5,
         10.0: 1,
         11.0: 1,
         12.0: 7,
         13.0: 2,
         14.0: 2,
         15.0: 2,
         16.0: 1,
         17.0: 1,
         19.0: 1,
         20.0: 2,
         21.0: 1,
         22.0: 1,
         23.0: 2,
         25.0: 1,
         27.0: 1,
         29.0: 2,
         32.0: 1,
         34.0: 2,
         35.0: 1})

In [92]:
Counter(sorted(q))

Counter({0.0: 1,
         1.0: 3,
         2.0: 4,
         3.0: 2,
         4.0: 1,
         6.0: 3,
         8.0: 1,
         9.0: 1,
         10.0: 2,
         12.0: 2,
         16.0: 2,
         17.0: 2,
         18.0: 1,
         19.0: 1,
         21.0: 2,
         23.0: 2,
         24.0: 2,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 1,
         33.0: 2,
         34.0: 1,
         35.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         44.0: 1,
         48.0: 1,
         49.0: 1})

In [93]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [21, 25, 57, 64, 68, 69, 80, 83, 98, 114, 119, 125, 130, 136, 142, 153, 159, 177]
1: [4, 12, 21, 25, 29, 36, 37, 57, 64, 67, 68, 69, 74, 78, 80, 83, 91, 97, 98, 108, 114, 117, 118, 119, 125, 130, 136, 142, 153, 155, 159, 177]
3: [2, 4, 10, 12, 13, 21, 22, 25, 27, 28, 29, 36, 37, 42, 57, 60, 64, 67, 68, 69, 72, 74, 75, 78, 80, 82, 83, 88, 91, 92, 93, 94, 95, 96, 97, 98, 102, 104, 108, 109, 110, 111, 114, 117, 118, 119, 121, 123, 125, 127, 130, 136, 137, 142, 147, 148, 153, 154, 155, 157, 159, 160, 163, 164, 165, 167, 177, 183]
NA: [6, 16, 24, 34, 35, 41, 46, 47, 49, 50, 54, 61, 63, 65, 66, 71, 77, 89, 90, 99, 100, 105, 106, 107, 122, 126, 128, 139, 141, 144, 145, 151, 162, 166, 169, 170, 172, 173, 174, 176, 178, 180, 181, 187, 188, 190]


In [94]:
AA = testing_2019.copy()
AA['Prediction'] = np.round(predictions)
AA['Difference'] = AA.SixthYear_HR - AA.Prediction

In [103]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = BBB.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2019)))
print("正負1：%.4f" % (a/len(y_test_2019)))
print("正負3：%.4f" % (b/len(y_test_2019)))
print("正負5：%.4f" % (c/len(y_test_2019)))
print("正負10：%.4f" % (d/len(y_test_2019)))

答對：0.1257
正負1：0.2094
正負3：0.3822
正負5：0.5393
正負10：0.7696


In [104]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.392659023794204
mean_squared_error: 73.43779015062353
rmse: 8.569585179611877


In [105]:
Counter(sorted(p))

Counter({3.0: 3,
         5.0: 1,
         7.0: 2,
         8.0: 1,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         12.0: 4,
         13.0: 1,
         14.0: 1,
         15.0: 2,
         16.0: 1,
         17.0: 1,
         18.0: 2,
         27.0: 1,
         29.0: 1})

In [106]:
Counter(sorted(m))

Counter({0.0: 1,
         1.0: 3,
         2.0: 4,
         3.0: 3,
         4.0: 1,
         5.0: 5,
         6.0: 2,
         7.0: 4,
         8.0: 3,
         9.0: 7,
         10.0: 2,
         11.0: 6,
         12.0: 8,
         13.0: 4,
         14.0: 3,
         15.0: 2,
         16.0: 1,
         17.0: 3,
         18.0: 2,
         20.0: 3,
         21.0: 1,
         27.0: 1,
         29.0: 1,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [107]:
Counter(sorted(n))

Counter({2.0: 2,
         3.0: 3,
         5.0: 2,
         6.0: 1,
         7.0: 2,
         8.0: 1,
         9.0: 3,
         10.0: 2,
         11.0: 2,
         12.0: 5,
         13.0: 3,
         14.0: 2,
         15.0: 2,
         16.0: 1,
         17.0: 2,
         18.0: 2,
         20.0: 1,
         21.0: 1,
         27.0: 1,
         29.0: 1,
         32.0: 1})

In [108]:
Counter(sorted(q))

Counter({0.0: 1,
         1.0: 3,
         2.0: 3,
         3.0: 2,
         4.0: 2,
         6.0: 2,
         12.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 1,
         19.0: 1,
         21.0: 2,
         23.0: 1,
         24.0: 3,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 2,
         33.0: 3,
         34.0: 2,
         35.0: 2,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         41.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [109]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [13, 20, 21, 25, 29, 36, 57, 59, 72, 74, 78, 91, 97, 108, 110, 113, 121, 123, 131, 146, 153, 158, 165, 177]
1: [9, 12, 13, 20, 21, 25, 29, 36, 44, 57, 59, 60, 67, 69, 72, 74, 78, 80, 91, 97, 101, 104, 108, 110, 111, 113, 119, 121, 123, 124, 125, 130, 131, 137, 146, 153, 158, 165, 177, 185]
3: [2, 4, 7, 9, 11, 12, 13, 20, 21, 22, 23, 25, 27, 29, 36, 42, 44, 48, 55, 56, 57, 59, 60, 64, 67, 68, 69, 72, 73, 74, 75, 78, 80, 83, 86, 91, 95, 96, 97, 101, 102, 103, 104, 108, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 124, 125, 130, 131, 137, 142, 146, 147, 148, 153, 157, 158, 159, 160, 165, 167, 177, 184, 185]
NA: [6, 15, 24, 26, 34, 35, 38, 46, 47, 49, 54, 61, 63, 65, 66, 71, 79, 89, 99, 100, 105, 106, 107, 126, 128, 129, 141, 144, 145, 155, 156, 162, 166, 169, 170, 172, 173, 174, 178, 179, 180, 181, 182, 187]


In [110]:
BB = testing_2019.copy()
BB['Prediction'] = np.round(predictions)
BB['Difference'] = BB.SixthYear_HR - BB.Prediction

In [111]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = CCC.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2019)))
print("正負1：%.4f" % (a/len(y_test_2019)))
print("正負3：%.4f" % (b/len(y_test_2019)))
print("正負5：%.4f" % (c/len(y_test_2019)))
print("正負10：%.4f" % (d/len(y_test_2019)))

答對：0.0995
正負1：0.2147
正負3：0.4031
正負5：0.5602
正負10：0.7958


In [112]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.138255634857098
mean_squared_error: 68.37440208419524
rmse: 8.268881549774143


In [113]:
Counter(sorted(p))

Counter({2.0: 3,
         4.0: 1,
         6.0: 1,
         7.0: 2,
         8.0: 2,
         12.0: 5,
         17.0: 1,
         20.0: 1,
         21.0: 1,
         23.0: 1,
         24.0: 1})

In [114]:
Counter(sorted(n))

Counter({1.0: 1,
         2.0: 5,
         3.0: 3,
         4.0: 1,
         6.0: 1,
         7.0: 2,
         8.0: 2,
         9.0: 2,
         10.0: 2,
         11.0: 1,
         12.0: 6,
         13.0: 1,
         14.0: 1,
         15.0: 2,
         17.0: 2,
         18.0: 2,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         23.0: 1,
         24.0: 1,
         27.0: 1,
         32.0: 1})

In [115]:
Counter(sorted(m))

Counter({0.0: 2,
         1.0: 4,
         2.0: 6,
         3.0: 4,
         4.0: 1,
         5.0: 6,
         6.0: 3,
         7.0: 3,
         8.0: 2,
         9.0: 4,
         10.0: 3,
         11.0: 5,
         12.0: 8,
         13.0: 1,
         14.0: 3,
         15.0: 2,
         16.0: 2,
         17.0: 2,
         18.0: 2,
         20.0: 2,
         21.0: 1,
         22.0: 3,
         23.0: 2,
         24.0: 1,
         27.0: 1,
         29.0: 1,
         31.0: 1,
         32.0: 1,
         34.0: 1})

In [116]:
Counter(sorted(q))

Counter({0.0: 1,
         1.0: 3,
         2.0: 4,
         3.0: 2,
         4.0: 1,
         6.0: 1,
         9.0: 1,
         10.0: 1,
         12.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         23.0: 1,
         24.0: 3,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 1,
         33.0: 3,
         35.0: 2,
         37.0: 1,
         39.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [117]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [7, 12, 21, 29, 43, 57, 60, 64, 67, 75, 80, 110, 119, 149, 159, 163, 165, 177, 184]
1: [7, 12, 20, 21, 25, 28, 29, 36, 42, 43, 57, 60, 64, 67, 69, 72, 74, 75, 78, 80, 91, 103, 110, 111, 113, 114, 119, 123, 125, 131, 142, 146, 148, 149, 153, 159, 163, 165, 168, 177, 184]
3: [2, 4, 5, 7, 9, 11, 12, 13, 18, 20, 21, 22, 23, 25, 28, 29, 32, 36, 39, 42, 43, 44, 57, 60, 64, 67, 68, 69, 72, 74, 75, 78, 79, 80, 88, 91, 92, 93, 94, 95, 96, 97, 101, 103, 104, 108, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 125, 131, 136, 142, 146, 147, 148, 149, 153, 154, 157, 159, 160, 163, 164, 165, 168, 175, 177, 184, 185]
NA: [6, 15, 24, 34, 35, 47, 49, 52, 54, 63, 65, 66, 71, 89, 99, 100, 106, 107, 122, 126, 128, 129, 139, 141, 145, 151, 166, 169, 170, 172, 173, 174, 178, 179, 180, 181, 182, 187, 190]


In [118]:
CC = testing_2019.copy()
CC['Prediction'] = np.round(predictions)
CC['Difference'] = CC.SixthYear_HR - CC.Prediction

In [119]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = DDD.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2019)))
print("正負1：%.4f" % (a/len(y_test_2019)))
print("正負3：%.4f" % (b/len(y_test_2019)))
print("正負5：%.4f" % (c/len(y_test_2019)))
print("正負10：%.4f" % (d/len(y_test_2019)))

答對：0.1099
正負1：0.2147
正負3：0.3979
正負5：0.5079
正負10：0.7435


In [120]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.97267770517559
mean_squared_error: 90.18605019370817
rmse: 9.496633624274876


In [121]:
Counter(sorted(p))

Counter({2.0: 5,
         5.0: 1,
         6.0: 2,
         7.0: 2,
         8.0: 1,
         9.0: 3,
         12.0: 2,
         13.0: 2,
         18.0: 1,
         32.0: 1,
         35.0: 1})

In [122]:
Counter(sorted(n))

Counter({1.0: 4,
         2.0: 6,
         3.0: 3,
         4.0: 1,
         5.0: 1,
         6.0: 2,
         7.0: 5,
         8.0: 1,
         9.0: 3,
         12.0: 3,
         13.0: 2,
         14.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         22.0: 1,
         29.0: 2,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [123]:
Counter(sorted(m))

Counter({0.0: 3,
         1.0: 5,
         2.0: 6,
         3.0: 5,
         4.0: 1,
         5.0: 6,
         6.0: 3,
         7.0: 5,
         8.0: 2,
         9.0: 4,
         10.0: 2,
         11.0: 1,
         12.0: 9,
         13.0: 2,
         14.0: 1,
         15.0: 2,
         16.0: 1,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         20.0: 2,
         21.0: 1,
         22.0: 2,
         23.0: 3,
         24.0: 1,
         25.0: 1,
         29.0: 2,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [124]:
Counter(sorted(q))

Counter({1.0: 2,
         2.0: 4,
         3.0: 2,
         6.0: 3,
         8.0: 1,
         10.0: 1,
         12.0: 1,
         14.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 2,
         18.0: 1,
         19.0: 2,
         21.0: 2,
         22.0: 1,
         23.0: 3,
         24.0: 3,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 2,
         33.0: 3,
         34.0: 2,
         35.0: 2,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         44.0: 1,
         48.0: 1,
         49.0: 1})

In [125]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [21, 22, 25, 55, 67, 83, 90, 94, 102, 111, 114, 118, 119, 130, 131, 136, 142, 147, 148, 171, 184]
1: [4, 10, 13, 18, 21, 22, 25, 42, 53, 55, 57, 59, 60, 67, 74, 78, 83, 88, 90, 91, 93, 94, 98, 102, 111, 114, 118, 119, 130, 131, 136, 142, 147, 148, 153, 157, 159, 167, 171, 177, 184]
3: [1, 2, 4, 5, 10, 13, 18, 21, 22, 25, 29, 32, 36, 37, 42, 45, 48, 53, 55, 57, 59, 60, 67, 69, 72, 74, 75, 77, 78, 80, 82, 83, 84, 88, 90, 91, 93, 94, 95, 96, 98, 102, 104, 108, 110, 111, 114, 118, 119, 121, 125, 127, 130, 131, 134, 136, 137, 140, 142, 146, 147, 148, 149, 150, 152, 153, 157, 159, 160, 163, 167, 168, 171, 175, 177, 184]
NA: [6, 15, 16, 24, 31, 34, 41, 46, 47, 50, 52, 54, 58, 61, 63, 65, 66, 71, 79, 86, 89, 99, 100, 105, 106, 107, 115, 126, 128, 129, 139, 141, 144, 145, 151, 155, 162, 166, 169, 170, 172, 173, 174, 176, 180, 181, 182, 187, 188]


In [126]:
DD = testing_2019.copy()
DD['Prediction'] = np.round(predictions)
DD['Difference'] = DD.SixthYear_HR - DD.Prediction

In [127]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = EEE.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2019)))
print("正負1：%.4f" % (a/len(y_test_2019)))
print("正負3：%.4f" % (b/len(y_test_2019)))
print("正負5：%.4f" % (c/len(y_test_2019)))
print("正負10：%.4f" % (d/len(y_test_2019)))

答對：0.1152
正負1：0.2147
正負3：0.3822
正負5：0.5183
正負10：0.8010


In [128]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.5714899906937365
mean_squared_error: 79.37488064052135
rmse: 8.909258141984738


In [129]:
Counter(sorted(p))

Counter({2.0: 1,
         3.0: 2,
         4.0: 1,
         5.0: 1,
         6.0: 1,
         7.0: 2,
         10.0: 1,
         11.0: 2,
         12.0: 4,
         13.0: 1,
         15.0: 1,
         16.0: 2,
         21.0: 1,
         23.0: 1,
         29.0: 1})

In [130]:
Counter(sorted(n))

Counter({0.0: 1,
         1.0: 1,
         2.0: 4,
         3.0: 2,
         4.0: 1,
         5.0: 2,
         6.0: 2,
         7.0: 2,
         8.0: 2,
         9.0: 3,
         10.0: 2,
         11.0: 2,
         12.0: 7,
         13.0: 2,
         15.0: 2,
         16.0: 2,
         17.0: 1,
         21.0: 1,
         23.0: 1,
         29.0: 1})

In [131]:
Counter(sorted(m))

Counter({0.0: 3,
         1.0: 4,
         2.0: 6,
         3.0: 3,
         4.0: 1,
         5.0: 5,
         6.0: 3,
         7.0: 4,
         8.0: 2,
         9.0: 5,
         10.0: 2,
         11.0: 3,
         12.0: 10,
         13.0: 3,
         14.0: 2,
         15.0: 3,
         16.0: 2,
         17.0: 3,
         18.0: 2,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         23.0: 2,
         27.0: 1,
         29.0: 1})

In [132]:
Counter(sorted(q))

Counter({1.0: 2,
         2.0: 5,
         3.0: 2,
         16.0: 1,
         17.0: 1,
         21.0: 2,
         23.0: 1,
         24.0: 3,
         26.0: 2,
         27.0: 1,
         28.0: 1,
         31.0: 2,
         33.0: 3,
         34.0: 2,
         35.0: 2,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         41.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [133]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [7, 10, 21, 23, 36, 57, 60, 67, 74, 77, 91, 97, 108, 113, 118, 121, 130, 152, 159, 165, 177, 183]
1: [7, 9, 10, 12, 21, 23, 25, 29, 36, 42, 50, 55, 57, 60, 67, 69, 74, 75, 77, 80, 91, 95, 97, 108, 110, 113, 114, 118, 119, 121, 130, 142, 146, 148, 152, 159, 165, 175, 177, 183, 184]
3: [2, 4, 7, 9, 10, 12, 18, 20, 21, 22, 23, 25, 29, 33, 36, 42, 48, 50, 55, 57, 60, 67, 69, 72, 73, 74, 75, 77, 78, 80, 82, 87, 88, 90, 91, 93, 94, 95, 96, 97, 98, 102, 104, 108, 110, 113, 114, 118, 119, 121, 130, 131, 137, 140, 142, 146, 148, 152, 153, 154, 157, 158, 159, 160, 164, 165, 167, 171, 175, 177, 183, 184, 185]
NA: [6, 15, 19, 24, 34, 38, 47, 54, 61, 63, 65, 66, 71, 79, 81, 89, 99, 100, 106, 107, 126, 128, 129, 141, 144, 151, 155, 162, 166, 169, 170, 172, 173, 174, 179, 180, 182, 187]


In [134]:
EE = testing_2019.copy()
EE['Prediction'] = np.round(predictions)
EE['Difference'] = EE.SixthYear_HR - EE.Prediction

In [189]:
BB[(BB.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [190]:
DD[(DD.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [191]:
EE[(EE.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [192]:
AA[(AA.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [194]:
CC[(CC.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference
